In [ ]:
import gym
import time
import math
import numpy as np
import random


In [ ]:
env = gym.make("CartPole-v1")

In [ ]:
bucketno = (6, 3)
totalAction = env.action_space.n
lower_bound = [env.observation_space.low[2], -math.radians(50)]
upper_bound = [env.observation_space.high[2], math.radians(50)]
q_table = np.zeros(bucketno + (totalAction,))
alpha = 0.9
gamma = 0.2
epsilon = 0.9
max_episode = 200

In [ ]:
from typing import Tuple
from sklearn.preprocessing import KBinsDiscretizer

def discretizer(_, __, angle, pole_velocity):
    kbin = KBinsDiscretizer(n_bins=bucketno, encode="ordinal", strategy="uniform")
    kbin.fit([lower_bound, upper_bound])
    return tuple(map(int, (kbin.transform([[angle, pole_velocity]])[0])))

In [ ]:
def action_selection(state, epsilon):
    if random.random() < epsilon:
        action = env.action_space.sample()
    else:
        action = np.argmax(q_table[state])
    return action

In [ ]:
def q_value_update(old_state, new_state, reward, action, alpha, gamma):
    future_q_value = np.argmax(q_table[new_state])
    old_state_value = q_table[old_state][action]
    q_table[old_state] += alpha*(reward + gamma * future_q_value - old_state_value)

In [ ]:

for v in range(max_episode):
    obs = [f for f in env.reset()[0]]
    initial_state = discretizer(*obs)
    current_state = initial_state
    total_reward = 0
    done = False
    while done == False:
        action = action_selection(current_state, epsilon)
        obs, reward, done, info, truncated = env.step(action)
        new_state = discretizer(*obs)
        q_value_update(current_state, new_state, reward, action, alpha, gamma)
        current_state = new_state
        env.render()
        total_reward+= reward
        print(total_reward)
env.close()
